In [1]:
from enum import Enum
class Memory(Enum):
    left = 0
    forward = 1
    right = 2
    
    def increment(self, inc=1):
        return Memory((self.value + inc) % 3)

class Direction(Enum):
    right = 0
    down = 1
    left = 2
    up = 3
    
    def to_position_increment(self):
        return [(1, 0), (0, 1), (-1, 0), (0, -1)][self.value]

    def increment(self, inc=1):
        return Direction((self.value + inc) % 4)
            
class TrackEvent(Enum):
    intersection = 0
    forward = 1
    back = 2

In [2]:
# Cart class
class Cart:
    def __init__(self, position, direction):
        self.position = position
        self.direction = direction
        self.memory = Memory.left
    
    def step(self, tracks):
        (x, y) = self.position
        (dx, dy) = self.direction.to_position_increment()
        self.position = (x + dx, y + dy)
        if self.position in tracks:
            event = tracks[self.position]
            self.turn(event)
    
    def turn(self, event):
        if event == TrackEvent.intersection:
            self.direction =  self.direction.increment(self.memory.value - 1)
            self.memory = self.memory.increment()
        elif event == TrackEvent.forward:
            self.direction = Direction([3, 2, 1, 0][self.direction.value])
        elif event == TrackEvent.back:
            self.direction = Direction([1, 0, 3, 2][self.direction.value])

In [3]:
from operator import itemgetter

class Collision(Exception):
    def __init__(self, coordinates):
        self.coordinates = coordinates
        
# Tick
def tick(carts, tracks):
    next_carts = {}
    for pos in sorted(carts.keys(), key = itemgetter(1, 0)):
        cart = carts.pop(pos)
        cart.step(tracks)
        if cart.position in carts or cart.position in next_carts:
            # Collision!
            raise Collision(cart.position)
        next_carts[cart.position] = cart
    return next_carts

In [4]:
# Input
carts = {}
tracks = {}
with open("Input/13.txt") as file:
    for y, line in enumerate(file):
        for x, char in enumerate(line):
            if char == '/':
                tracks[(x, y)] = TrackEvent.forward
            elif char == '\\':
                tracks[(x, y)] = TrackEvent.back
            elif char == '+':
                tracks[(x, y)] = TrackEvent.intersection
            elif char == '>':
                carts[(x, y)] = Cart((x, y), Direction.right)
            elif char == 'v':
                carts[(x, y)] = Cart((x, y), Direction.down)
            elif char == '<':
                carts[(x, y)] = Cart((x, y), Direction.left)
            elif char == '^':
                carts[(x, y)] = Cart((x, y), Direction.up)

In [5]:
# Part 1

while True:
    try:
        carts = tick(carts, tracks)
    except Collision as c:
        (x, y) = c.coordinates
        print("{},{}".format(x, y))
        break

113,136


In [6]:
# Reset input
carts = {}
tracks = {}
with open("Input/13.txt") as file:
    for y, line in enumerate(file):
        for x, char in enumerate(line):
            if char == '/':
                tracks[(x, y)] = TrackEvent.forward
            elif char == '\\':
                tracks[(x, y)] = TrackEvent.back
            elif char == '+':
                tracks[(x, y)] = TrackEvent.intersection
            elif char == '>':
                carts[(x, y)] = Cart((x, y), Direction.right)
            elif char == 'v':
                carts[(x, y)] = Cart((x, y), Direction.down)
            elif char == '<':
                carts[(x, y)] = Cart((x, y), Direction.left)
            elif char == '^':
                carts[(x, y)] = Cart((x, y), Direction.up)

In [7]:
# Part 2
def tick_safe(carts, tracks):
    next_carts = {}
    for pos in sorted(carts.keys(), key = itemgetter(1, 0)):
        cart = carts.pop(pos, None)
        if not cart:
            # This cart probably collided during this tick
            continue
        cart.step(tracks)
        if cart.position in carts or cart.position in next_carts:
            # Collision! We remove the carts this time
            carts.pop(cart.position, None)
            next_carts.pop(cart.position, None)
            continue
        next_carts[cart.position] = cart
    return next_carts

while len(carts) > 1:
    carts = tick_safe(carts, tracks)

for (x, y) in carts.keys():
    print("{},{}".format(x, y))


114,136
